### Setup

1. Make a copy of `../.env.example` to `../.env` and fill the `OPENAI_API_KEY` and `CHAINLIT_API_KEY` environment variables.
2. This demo assumes that chainlit-cloud is running locally.
3. Run the cells to install the dependencies and the local sdk package.

In [ ]:
pip install -r ../requirements.txt

In [ ]:
pip install -e ..

### Example

In [ ]:
import asyncio
import json
import uuid

from chainlit_sdk import Chainlit
from chainlit_sdk.observability_agent import OperatorRole
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI()

sdk = Chainlit(batch_size=2)
sdk.instrument_openai()

In [ ]:
# Start a conversation

thread_id = uuid.uuid4()


@sdk.observer.message(thread_id=thread_id)
def send_message(text):
    print(text)
    step = sdk.observer.get_current_step()
    step.output = text
    step.operatorRole = OperatorRole.ASSISTANT


@sdk.observer.run(thread_id=thread_id)
def get_completion(input):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Tell an inspiring quote to the user, mentioning their name. Be extremely supportive while keeping it short. Write one sentence per line.",
            },
            {
                "role": "assistant",
                "content": welcome_message,
            },
            {
                "role": "user",
                "content": text,
            },
        ],
    )
    return completion


welcome_message = "What's your name? "
with sdk.observer.step(type="message", thread_id=thread_id) as step:
    step.output = welcome_message
    step.operatorRole = OperatorRole.SYSTEM

text = input(welcome_message)

with sdk.observer.step(type="message", thread_id=thread_id) as step:
    step.output = text
    step.operatorRole = OperatorRole.USER
    message = get_completion(text)
    send_message(message.choices[0].message.content)

In [ ]:
sdk.wait_until_queue_empty() # wait for all msgs to be in the cloud instance

In [ ]:
# Get the thread

print("\nSearching for the thread", thread_id, "...")
steps = await sdk.api.get_thread(id=str(thread_id))

thread = steps

print(json.dumps(thread, indent=2))